In [ ]:
from carbon import c12, decoding
from typing import Sequence
from collections import Counter
from errata.pauli import SparsePauli as Pauli
import qsharp

# Initialize Q# with the project and adaptive profile configured
qsharp.init(target_profile=qsharp.TargetProfile.Adaptive_RIF, project_root=".")

In [ ]:
# Set up the C12 decoders

# Create a lookup table for the code that will be used for converting
# syndromes to recoveries.
table = decoding.make_phenomenological_table(max_css_weight=1)

# Utility for converting a Q# syndrome measurement into a Pauli for use in
# recovery logic.
def results_as_pauli(results: Sequence[qsharp.Result], pauli: str = "Z") -> Pauli:
    p = ""
    for r in results:
        if r == qsharp.Result.One:
            p += pauli
        else:
            p += "I"
    return Pauli.from_string(p)

# Utility or converting a pauli to a Q# result array assuming the given basis
# and count. Pauli I and the given basis are treated as Zero, remaining Pauli
# characters are interpreted as One.
def pauli_as_results(p: Pauli, basis: str = "Z", count: int = 2):
    results = []
    chars = "XYZ".replace(basis, "")
    for i in range(count):
        if p[i] in chars:
            results.append(qsharp.Result.One)
        else:
            results.append(qsharp.Result.Zero)
    return results

# Given an X syndrome measurement and a Z syndrome measurement,
# calculate the logical recovery for the combined syndrome, using the global
# lookup table.
def recovery_from_syndrome_measurements(x_meas: Sequence[qsharp.Result], z_meas: Sequence[qsharp.Result]) -> Pauli:
    error_z = results_as_pauli(x_meas, pauli="Z")
    error_x = results_as_pauli(z_meas, pauli="X")
    error = error_z * error_x
    syndrome = frozenset(c12.code.syndrome_of(error))
    recovery = table.get(syndrome, None)
    return abs(c12.code.logical_action_of(recovery * error))

# For each tuple of shot results, mark the shot as preselect if any preselect
# measurement is One. Then use the rounds of error correction syndrome measurements
# to generate Pauli corrections, collected in a frame. Use the final Pauli frame
# to calculate the corrected logical results.
def decode_results(results, basis: str = "Z"):
    corrected_logical_results = []
    for res in results:
        corrected_logical_results.append([])
        for shot in res:
            if any([preselect != qsharp.Result.Zero for preselect in shot[0]]):
                corrected_logical_results += ["PREselect"]
                continue
            recovery = Pauli.identity()
            r = None
            for ec_output in shot[1]:
                r = recovery_from_syndrome_measurements(ec_output[0], ec_output[1])
                if r is None:
                    corrected_logical_results += ["POSTselect"]
                    break
                recovery *= r
            if r is None:
                corrected_logical_results += [pauli_as_results(recovery, basis=basis)]
                continue
            if basis == "Z":
                r = recovery_from_syndrome_measurements([], shot[2])
            else:
                assert basis == "X"
                r = recovery_from_syndrome_measurements(shot[2], [])
            if r is None:
                corrected_logical_results += ["POSTselect"]
                continue
            recovery *= r
            corrected_logical_results[-1] += [pauli_as_results(recovery, basis=basis)]
    return corrected_logical_results

In [ ]:
%%qsharp
import Std.Diagnostics.Fact;
import Utils.TransversalCNOT;
import C12;

operation PerformTeleportExperiment(ec_repetitions : Int, basis : Pauli, num_blocks : Int) : (Result[], (Result[], Result[])[], Result[])[] {
    Fact(num_blocks > 0, "need at least one block");
    use qubits = Qubit[(12 + 16) * num_blocks];
    mutable results = [];
    for i in 0..(num_blocks - 1) {
        let logical_block = qubits[(0+(i*28))..(11+(i*28))];
        let ancillas = qubits[(12+(i*28))..(27+(i*28))];
        // The last 8 qubits are padding.

        results += [C12_Mark2_1Q_Teleport(ec_repetitions, basis, logical_block, ancillas)];
    }
    return results;
}

operation C12_Mark2_1Q_Teleport(ec_repetitions : Int, basis : Pauli, logical_block : Qubit[], ancillas : Qubit[]) : (Result[], (Result[], Result[])[], Result[]) {
    Fact(basis == PauliZ or basis == PauliX or basis == PauliI, "only PauliZ and PauliX supported");
    Fact(Length(logical_block) == 12, "logical block must be 12 qubits");
    Fact(Length(ancillas) == 16, "need 16 ancillas");

    // Prepare in the requested basis
    mutable preselect = if basis == PauliX {
        C12.PrepareXX(logical_block, ancillas[...3])
    } else {
        C12.PrepareZZ(logical_block, ancillas[...3])
    };

    mutable syndromes = [];

    for _ in 1..(ec_repetitions) {
        // Sequential teleport on..
        // Prepare Z, Teleport X
        set preselect += C12.PrepareZZ(ancillas[...11], ancillas[12...]);
        TransversalCNOT(logical_block, ancillas[...11]);
        ApplyToEach(H, logical_block);
        let syndrome_x = MResetEachZ(logical_block);

        // Prepare X, Teleport Z
        set preselect += C12.PrepareXX(logical_block, ancillas[12...]);
        TransversalCNOT(logical_block, ancillas[...11]);
        let syndrome_z = MResetEachZ(ancillas[...11]);
        set syndromes += [(syndrome_x, syndrome_z)];
    }

    // Final measurement
    if basis == PauliX {
        ApplyToEach(H, logical_block);
    }
    let final = MResetEachZ(logical_block);

    (preselect, syndromes, final)
}

In [ ]:
results = qsharp.run(qsharp.code.PerformTeleportExperiment, 100, 1, qsharp.Pauli.Z, 1)
corrected_logical_results = decode_results(results, "Z")

# Use counter to make a simple histogram of the corrected logical results.
from qsharp_widgets import Histogram
Histogram(map(str, corrected_logical_results))

In [ ]:
qir = qsharp.compile(qsharp.code.PerformTeleportExperiment, 1, qsharp.Pauli.Z, 1)
len(str(qir).splitlines())

In [ ]:
from qsharp.passes import transform

transformed_qir = transform(qir, verbose=True)
len(str(transformed_qir).splitlines())

In [ ]:
from qsharp.passes import trace, AC1K
from qsharp_widgets import Atoms

Atoms(AC1K, trace(transformed_qir))

In [ ]:
from qsharp.passes import transform_to_clifford
clifford_qir = transform_to_clifford(qir)
len(str(clifford_qir).splitlines())

In [ ]:
from qsharp._simulation import run_qir, NoiseConfig
from qsharp_widgets import Histogram

res = run_qir(str(clifford_qir), shots=100)
corrected_logical_results = decode_results(res, "Z")
Histogram(map(str, corrected_logical_results))

In [ ]:
noise = NoiseConfig()
noise.sx.loss = 0.001

res = run_qir(str(clifford_qir), shots=100, noise=noise)
corrected_logical_results = decode_results(res, "Z")
Histogram(map(str, corrected_logical_results))